# Decision Trees

non parametric model that divides the predictor space into a number of simpler regions based on a number of subsequent decision rules

no parameters means that we have only hyperparameters to tune
no equation to write, no cost function to minimize

- tree starts with the entire training dataset in the root node
- the tree then divides the training data into two non-overlapping subsets by branching out into two child nodes
- every branch out is based on a decision rule
- new nodes of the tree keep branching out until a certain point
- a node that doesn't branch out anymore is a leaf node

a decision rule at every branching node consists of one of the predictors and a threshold value imposed on it

if we never stopped branching out, then we would only have pure nodes -- perfect training performance but terribly overfitting
need regularization to determine when to stop branching out

branching out - regression:
- MSE value is calcualted with the training responses and the average response of all the training observations from the root node
- the algorithm finds the predictor-threshold pair (decision rule) to split the observations, such that the weighted sum of the MSE at the child nodes are minimum
    - weighted sum means that the MSE is multiplied with the number of observations at the node
- after each branching out the total MSE of the model is the sum of MSE values of the outermost nodes
- so at each split the algo finds the predictor-threshold pair that decreases the total weighted MSE of the tree model the most
- the algo keeps finding new decision rules for new nodes to further decrease the total weighted MSE
- process has to stop at a certain point

branching out - classification:
- for classifcation the algo behaves the same way - the only deifference is the metric
- usual classification metrics of accuracy, precision, recall, etc are not used here --> aren't sensitive enough for a decision tree
    - you can split nodes without changing class balances, so no use in separating classes
- use a purity metric for each node: Gini Entropy

$$
G = \sum_{k=1}^{K} p_{mk} (1 - p_{mk})
$$

where \( p_{mk} \) is the proportion of observations from class \( k \) among all observations at node \( m \).


without any intervention, the trees would want to grow until all leaf nodes have 0 MSE/Gini

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import GridSearchCV, KFold


In [2]:
# training data
trainf = pd.read_csv("/Users/vaibhavrangan/Downloads/Stat_303-3/Datasets/Car_features_train.csv") # predictors
trainp = pd.read_csv("/Users/vaibhavrangan/Downloads/Stat_303-3/Datasets/Car_prices_train.csv") # response
train =  pd.merge(trainf, trainp)

# test data
testf = pd.read_csv("/Users/vaibhavrangan/Downloads/Stat_303-3/Datasets/Car_features_test.csv") # predictors
testp = pd.read_csv("/Users/vaibhavrangan/Downloads/Stat_303-3/Datasets/Car_prices_test.csv") # response
test = pd.merge(testf, testp)

predictors = ["mpg", "engineSize", "year", "mileage"]

X_train = train[predictors]
y_train = train["price"]

X_test = test[predictors]
y_test = test["price"]

No scaling necessary for decision trees

because the training algorthm uses one predictor at a time while giving a decision rule at a node
so, scaled or unscaled data is not important for the decision rule, the threshold will just be scaled

In [4]:
tree_model = DecisionTreeRegressor(
    random_state=12, # to be fixed not tuned
    max_depth = 5, # limits the number of levels (branching out) in the tree: FIRST HYPERPARAMETER TO TUNE
    max_leaf_nodes = 30 # limits the number of leaf nodes in the tree: SECOND HYPERPARAMETER TO TUNE
)

tree_model.fit(X_train, y_train)

y_pred = tree_model.predict(X_test)
root_mean_squared_error(y_test, y_pred)

8455.970586094883

# Tuning a Decision Tree

decision trees are the first model with two numeric hyperparameters

In [5]:
# The important thing to note here: with both hyperparam with nuemric ranges
    # the grid search can get too expensive

# We need to approach the grid searches efficiently:

# Start with the empty model
model = DecisionTreeRegressor(random_state=12) # random_state fixed, not tuned

# Note the large stepsizes in the grid -- this is coarse tuning
grid = {
    'max_depth': range(2, 30, 3),
    'max_leaf_nodes': range(2, 4900, 100)
}

# The reason for large stepsizes:
    # the search has less combinations to try -- runs faster
    # They give us the area in the grid to focus

cv = KFold(n_splits=5, shuffle=True, random_state=1)

gscv = GridSearchCV(model, grid, cv=cv, scoring='neg_root_mean_squared_error',verbose=1, n_jobs=-1)

gscv.fit(X_train, y_train)

Fitting 5 folds for each of 490 candidates, totalling 2450 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=DecisionTreeRegressor(random_state=12), n_jobs=-1,
             param_grid={'max_depth': range(2, 30, 3),
                         'max_leaf_nodes': range(2, 4900, 100)},
             scoring='neg_root_mean_squared_error', verbose=1)

In [6]:
print("Best hyperparameters: ", gscv.best_params_)
print("Best score: ", -gscv.best_score_)

Best hyperparameters:  {'max_depth': 11, 'max_leaf_nodes': 302}
Best score:  6462.682515529132


In [7]:
# Redo the grid search -- fine tuning this time!

model = DecisionTreeRegressor(random_state=12)

grid = {
    'max_depth': range(8, 14), # A finer grid centered around 11 (what we found above)
    'max_leaf_nodes': range(200, 410, 10) # A finer grid centered around 302 (what we found above)
}

# The rest is the same
cv = KFold(n_splits=5, shuffle=True, random_state=1)
gscv = GridSearchCV(model, grid, cv=cv, scoring='neg_root_mean_squared_error',verbose=1, n_jobs=-1)
gscv.fit(X_train, y_train)



Fitting 5 folds for each of 126 candidates, totalling 630 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=DecisionTreeRegressor(random_state=12), n_jobs=-1,
             param_grid={'max_depth': range(8, 14),
                         'max_leaf_nodes': range(200, 410, 10)},
             scoring='neg_root_mean_squared_error', verbose=1)

In [8]:
print(gscv.best_params_)
print(-gscv.best_score_)

{'max_depth': 9, 'max_leaf_nodes': 210}
6450.246390041281


In [9]:
# One final tuning -- most granular tuning

model = DecisionTreeRegressor(random_state=12)

grid = {
    'max_depth': range(8, 12), # Same range as before, narrowed down a bit
    'max_leaf_nodes': range(200, 230) # A finer grid centered around 302 (what we found above)
}

# The rest is the same
cv = KFold(n_splits=5, shuffle=True, random_state=1)
gscv = GridSearchCV(model, grid, cv=cv, scoring='neg_root_mean_squared_error',verbose=1, n_jobs=-1)
gscv.fit(X_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=DecisionTreeRegressor(random_state=12), n_jobs=-1,
             param_grid={'max_depth': range(8, 12),
                         'max_leaf_nodes': range(200, 230)},
             scoring='neg_root_mean_squared_error', verbose=1)

In [10]:
print("Best hyperparameters: ", gscv.best_params_)
print("Best score: ", -gscv.best_score_)

Best hyperparameters:  {'max_depth': 9, 'max_leaf_nodes': 215}
Best score:  6446.277103539869


This step-by-step approach of coarse tuning first, finer tuning afterwards in a few steps is very efficient, even with expensive models